
# Introduction to TensorFlow

scikit-learn does not offer Neural Network implementation libraries, whereas TensorFlow does

In [1]:
import tensorflow as tf
import sklearn
import numpy as np

/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Iris dataset

Load the dataset

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
print (iris.feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [17]:
X = iris.data
y = iris.target

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

Define input function

TensorFlow requires features in dictionary format

In [21]:
def input_evaluation(x,y):
    
    features = {'SepalLength': x[:,0],
            'SepalWidth': x[:,1],
            'PetalLength': x[:,2],
            'PetalWidth': x[:,3]}

    labels = y
    
    return features, labels

'''batch_size defines the number of data units for batch learning:
    batch_size = 1 : online learning
    1 < batch_size < len(X) : mini-batch learning
    batch_size = len(X) : gradient descent learning'''

batch_size = 1

TensorFlow requires the creation of a dataset and a dataset iterator which provides a way of accessing one dataset element at a time

More information here: https://www.tensorflow.org/get_started/datasets_quickstart

In [6]:
def train_input_fn(features, labels, batch_size):
    
    # From_tensor_slices separates the dictionary input into array slices.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle by creating a buffer (here length of 1000 > len(X)), repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [7]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)

    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

Note: a one-shot iterator is the simplest form of iterator, which only iterates once through the dataset and is currently the only type of iterator that is easily useable with an Estimator

Other iterators are available which can allow, e.g. more convenient switching between datasets

More information here: https://www.tensorflow.org/programmers_guide/datasets#creating_an_iterator

In [22]:
#Create TF dictionary inputs
X_trainTF, y_trainTF = input_evaluation(X_train, y_train)
X_testTF, y_testTF = input_evaluation(X_test, y_test)

'Feature columns' is a Google term used to describe the intemetiary structure created to bridge the gap between raw data and the Esitmator input. The user specifies the data type of the raw data and the output data. The feature columns function can be the equivalent of creating one-hot variables for the y values.

https://www.tensorflow.org/versions/master/get_started/feature_columns

In [9]:
# Feature columns describe how to use the input.
FEATURE_KEYS = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
feature_columns = [
      tf.feature_column.numeric_column(key, shape=1) for key in FEATURE_KEYS]

In [10]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp26_zgmi5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f41b96d2860>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
# Train the Model.
classifier.train(input_fn=lambda:train_input_fn(X_trainTF, y_trainTF, batch_size), steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp26_zgmi5/model.ckpt.
INFO:tensorflow:loss = 0.70999044, step = 1
INFO:tensorflow:global_step/sec: 436.262
INFO:tensorflow:loss = 0.021928001, step = 101 (0.231 sec)
INFO:tensorflow:global_step/sec: 559.587
INFO:tensorflow:loss = 0.6601869, step = 201 (0.179 sec)
INFO:tensorflow:global_step/sec: 381.643
INFO:tensorflow:loss = 0.4384018, step = 301 (0.262 sec)
INFO:tensorflow:global_step/sec: 430.641
INFO:tensorflow:loss = 0.18581378, step = 401 (0.232 sec)
INFO:tensorflow:global_step/sec: 576.299
INFO:tensorflow:loss = 0.29081342, step = 501 (0.174 sec)
INFO:tensorflow:global_step/sec: 438.081
INFO:tensorflow:loss = 0.0065913643, step = 601 (0.227 sec)
INFO:tensorflow:global_step/sec: 383.15
INFO:tensorflow:loss = 0.56136703, step = 701 (0.261 sec)
INFO:tensorflow:global_step/sec: 362.408
INFO:tensorflow:loss = 0.003543885, step = 801 (0.275 sec)
INFO:tensorflow:global_step/sec: 469.231
INF

In [12]:
# Evaluate the model.
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(X_testTF, y_testTF, batch_size))

INFO:tensorflow:Starting evaluation at 2018-03-03-17:42:39
INFO:tensorflow:Restoring parameters from /tmp/tmp26_zgmi5/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-03-03-17:42:39
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.92105263, average_loss = 0.15918867, global_step = 1000, loss = 0.15918867


In [13]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.921



## Model building

TensorFlow allows for increased dynamism in allowing users to build and alter models which can be used in place of the TensorFlow standard premade Estimators

In [14]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(X, y,batch_size=1, num_epochs=None, shuffle=True)

In [15]:
# Define the neural network
def neural_net(x_dict, n_hidden_1, n_hidden_2, num_classes):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']

    layer_1 = tf.layers.dense(x, n_hidden_1)

    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    
    return out_layer

In [16]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs